In [1]:
import pandas as pd
import os 
import numpy as np
import re

## 1. Import research collection data

In [2]:
# import relevant document types
books = pd.read_csv("books-2020-10-12.csv")
conf = pd.read_csv("conference-2020-10-12.csv")
journ = pd.read_csv("journal-2020-10-12.csv")
pap = pd.read_csv("papers-2020-10-12.csv")

/Users/andreasopedal/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (4,5,21,51,56,75,76,83,84,85,87,88) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/andreasopedal/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (5,35,49,52,54,56,57,72,73,74,77,79,86,87,91) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/andreasopedal/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (5,19,22,23,36,50,52,53,54,55,58,59,73,74,75,76,79,80,81,82,83,85,89,90,91,94,95,106,108) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/andreasopedal/opt/anaconda3/lib/pyt

In [3]:
research_data_df = pd.concat([books, conf, journ, pap], ignore_index=True)
print(research_data_df.shape)
research_data_df.head()

(176057, 114)


,id,collection,dc.contributor.author,dc.contributor.editor,dc.contributor.other,dc.contributor.supervisor,dc.date.accessioned,dc.date.available,dc.date.issued,dc.date.published,...,ethz.relation.isCompiledBy,ethz.relation.isDocumentedBy,ethz.relation.references,dc.language,ethz.identifier.pubmed,ethz.relation.isReviewedBy,ethz.relation.reviews,ethz.version,ethz.date.retentionend,ethz.date.retentionendDate
0,108069,NaN,"Schützeichel, Rainer","Gerber, Andri||Kurath, Stefan",NaN,NaN,2017-06-11T21:44:58Z,2017-06-11T21:44:58Z,2016,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,188444,NaN,"Milman, Vitali||Wagner, Roy","Ball, Keith M.||Milman, Vitali",NaN,NaN,2017-10-23T13:08:25Z,2017-06-12T14:30:31Z||2017-08-21T11:33:24Z||20...,1999-04,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,125569,NaN,"Folkers, Gerd","Abel, Günter||Plümacher, Martina",NaN,NaN,2017-06-12T18:11:46Z,2017-06-12T18:11:46Z,2016,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,157378,NaN,"Mack, Rüdiger||Schleich, Wolfgang P.||Haase, D...","Arendt, Wolfgang||Schleich, Wolfgang P.",NaN,NaN,2017-06-14T12:24:19Z,2017-06-14T12:24:19Z,2008,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,19843,NaN,"Burkhard, Remo||Schmitt, Gerhard","Wang, Xiangyu||Schnabel, Marc Aurel",NaN,NaN,2017-06-08T23:42:55Z,2017-06-08T23:42:55Z,2008,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
set(research_data_df.columns)

{'collection',
 'dc.contributor',
 'dc.contributor.author',
 'dc.contributor.editor',
 'dc.contributor.other',
 'dc.contributor.supervisor',
 'dc.date.accessioned',
 'dc.date.available',
 'dc.date.issued',
 'dc.date.published',
 'dc.description.abstract',
 'dc.format',
 'dc.identifier.doi',
 'dc.identifier.isbn',
 'dc.identifier.issn',
 'dc.identifier.olduri',
 'dc.identifier.other',
 'dc.identifier.uri',
 'dc.identifier.wos',
 'dc.language',
 'dc.language.iso',
 'dc.publisher',
 'dc.relation.isnodouble',
 'dc.relation.ispartof',
 'dc.relation.isreplacedbydouble',
 'dc.rights.license',
 'dc.rights.uri',
 'dc.subject',
 'dc.title',
 'dc.title.alternative',
 'dc.type',
 'ethz.availability',
 'ethz.book.title',
 'ethz.code.ddc',
 'ethz.code.jel',
 'ethz.date.deposited',
 'ethz.date.embargoend',
 'ethz.date.retentionend',
 'ethz.date.retentionendDate',
 'ethz.doipreview',
 'ethz.ecitpid',
 'ethz.ecolpid',
 'ethz.edit.source',
 'ethz.edit.status',
 'ethz.eth',
 'ethz.event',
 'ethz.event.da

In [5]:
# observe that none of the identifiers are complete
test = research_data_df[['dc.identifier.doi','dc.identifier.isbn','dc.identifier.issn']]#,'dc.identifier.olduri','dc.identifier.other','dc.identifier.uri',
                                             #'dc.identifier.wos']]
test.head()
notna = test.dropna(thresh=1)
notna.shape
# sum(research_data_df["dc.identifier.doi"].notna())
# sum(research_data_df["dc.identifier.isbn"].notna())
# sum(research_data_df["dc.identifier.issn"].notna())

(143905, 3)

143905/176057 have at least one identifier (isbn, issn, doi). IF olduri, other, uri and wos are added we only have 4 empty docs!

In [6]:
# select relevant columns (including all identifiers that might be useful)
research_data_df_sel = research_data_df[['dc.contributor.author','dc.description.abstract','dc.title','id', 
                                             'dc.identifier.doi','dc.identifier.isbn','dc.identifier.issn',
                                             'dc.identifier.olduri','dc.identifier.other','dc.identifier.uri',
                                             'dc.identifier.wos']]

In [7]:
# function for cleaning authors
def separate_names(names):
    """ Separes a string of names of the form name1||name2||name3||... into a list of names."""
    if isinstance(names, str):
        return names.split("||")

In [8]:
author_array = research_data_df_sel.loc[:,"dc.contributor.author"].apply(separate_names)
research_data_df_sel["dc.contributor.author"] = author_array
research_data_df_sel.head()

/Users/andreasopedal/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,dc.contributor.author,dc.description.abstract,dc.title,id,dc.identifier.doi,dc.identifier.isbn,dc.identifier.issn,dc.identifier.olduri,dc.identifier.other,dc.identifier.uri,dc.identifier.wos
0,"[Schützeichel, Rainer]",NaN,Die Perspektive des Städtebauers. Ein Blick au...,108069,NaN,978-3-86922-461-9||3-86922-461-4,NaN,NaN,NaN,http://hdl.handle.net/20.500.11850/108102,NaN
1,"[Milman, Vitali, Wagner, Roy]",The goal of this note is to introduce new clas...,Asymptotic versions for operators and operator...,188444,10.3929/ethz-b-000121674,978-0-521-64259-0||978-0-521-15564-9||0-521-64...,NaN,NaN,NaN,http://hdl.handle.net/20.500.11850/121674,NaN
2,"[Folkers, Gerd]",NaN,On Re-Positioning,125569,NaN,978-3-11-048264-5||978-3-11-049335-1||978-3-11...,2365-1601,NaN,NaN,http://hdl.handle.net/20.500.11850/125602,NaN
3,"[Mack, Rüdiger, Schleich, Wolfgang P., Haase, ...",NaN,Factorization,157378,NaN,978-3-527-40830-6||3-527-40830-4,NaN,NaN,NaN,http://hdl.handle.net/20.500.11850/157411,NaN
4,"[Burkhard, Remo, Schmitt, Gerhard]",NaN,Visualising future cities in the ETH Value Lab,19843,NaN,978-1-4020-9087-5||978-1-4020-9088-2,NaN,NaN,10.1007/978-1-4020-9088-2_13,http://hdl.handle.net/20.500.11850/19876,NaN


In [9]:
# explode on author
research_data_e = research_data_df_sel.explode("dc.contributor.author")
print(research_data_e.shape)

(867718, 11)


In [10]:
research_data_e.head()

,dc.contributor.author,dc.description.abstract,dc.title,id,dc.identifier.doi,dc.identifier.isbn,dc.identifier.issn,dc.identifier.olduri,dc.identifier.other,dc.identifier.uri,dc.identifier.wos
0,"Schützeichel, Rainer",NaN,Die Perspektive des Städtebauers. Ein Blick au...,108069,NaN,978-3-86922-461-9||3-86922-461-4,NaN,NaN,NaN,http://hdl.handle.net/20.500.11850/108102,NaN
1,"Milman, Vitali",The goal of this note is to introduce new clas...,Asymptotic versions for operators and operator...,188444,10.3929/ethz-b-000121674,978-0-521-64259-0||978-0-521-15564-9||0-521-64...,NaN,NaN,NaN,http://hdl.handle.net/20.500.11850/121674,NaN
1,"Wagner, Roy",The goal of this note is to introduce new clas...,Asymptotic versions for operators and operator...,188444,10.3929/ethz-b-000121674,978-0-521-64259-0||978-0-521-15564-9||0-521-64...,NaN,NaN,NaN,http://hdl.handle.net/20.500.11850/121674,NaN
2,"Folkers, Gerd",NaN,On Re-Positioning,125569,NaN,978-3-11-048264-5||978-3-11-049335-1||978-3-11...,2365-1601,NaN,NaN,http://hdl.handle.net/20.500.11850/125602,NaN
3,"Mack, Rüdiger",NaN,Factorization,157378,NaN,978-3-527-40830-6||3-527-40830-4,NaN,NaN,NaN,http://hdl.handle.net/20.500.11850/157411,NaN


## 2. Import organization and department data

### 2.1 Organization data

In [11]:
org_df = pd.read_excel("ETH Professor list.xlsx")
org_df.head()

,Title,Name,First name,Org. unit code,Organisation
0,Prof. Dr.,Abhari,Reza S.,2627,Institute of Energy Technology (former)
1,Prof. Dr.,Acciaio,Beatrice,9727,"Acciaio, Beatrice"
2,Prof. Dr.,Ackermann,Martin,3743,"Ackermann, Martin"
3,Prof. Dr.,Ackermann,Martin,2721,Inst. Biogeochem. and Pollutant Dynamics
4,Prof. Dr.,Adey,Bryan T.,2604,Inst. Construction&Infrastructure Manag.


In [12]:
# create professor column and keep only the relevant columns
org_df["Professor"] = org_df["Name"] + ", " + org_df["First name"]
org_df = org_df[["Professor", "Org. unit code", "Organisation"]]
org_df.head()

,Professor,Org. unit code,Organisation
0,"Abhari, Reza S.",2627,Institute of Energy Technology (former)
1,"Acciaio, Beatrice",9727,"Acciaio, Beatrice"
2,"Ackermann, Martin",3743,"Ackermann, Martin"
3,"Ackermann, Martin",2721,Inst. Biogeochem. and Pollutant Dynamics
4,"Adey, Bryan T.",2604,Inst. Construction&Infrastructure Manag.


In [13]:
# outer merge with research collection 
rc_m = org_df.merge(research_data_e,how="outer",left_on="Professor",right_on="dc.contributor.author")
rc_m.head(3)

,Professor,Org. unit code,Organisation,dc.contributor.author,dc.description.abstract,dc.title,id,dc.identifier.doi,dc.identifier.isbn,dc.identifier.issn,dc.identifier.olduri,dc.identifier.other,dc.identifier.uri,dc.identifier.wos
0,"Abhari, Reza S.",2627.0,Institute of Energy Technology (former),"Abhari, Reza S.",NaN,Non-axisymmetric end wall profiles including f...,20325.0,NaN,978-3-85125-036-7,NaN,NaN,NaN,http://hdl.handle.net/20.500.11850/20358,NaN
1,"Abhari, Reza S.",2627.0,Institute of Energy Technology (former),"Abhari, Reza S.",NaN,Stator-clocking effects on the unsteady intera...,8397.0,NaN,NaN,0957-6509||0954-4046||0263-7138||2041-2967,NaN,10.1243/09576509JPE458,http://hdl.handle.net/20.500.11850/8430,NaN
2,"Abhari, Reza S.",2627.0,Institute of Energy Technology (former),"Abhari, Reza S.",NaN,High Temperature Fast Response Aerodynamic Probe,159307.0,NaN,978-0-7918-4398-7,NaN,NaN,10.1115/GT2010-23010,http://hdl.handle.net/20.500.11850/159340,NaN


In [14]:
rc_m.shape

(877074, 14)

In [15]:
# remove entries without an organization
rc_m_2 = rc_m[rc_m['Organisation'].notna()]
print(rc_m_2.shape)

(80366, 14)


### 2.2 Department data

In [17]:
dep_df = pd.read_excel("Research areas from researcher profile page.xls")
dep_df.head()

,ANREDE,FAMNAME,VORNAME,DEPARTEMENT_NAME,DEPARTEMENT_LEITZAHL,FORSCHUNGSGEBIET_E,HOMEPAGE
0,Herr,Avermaete,Tom,Architektur,2100,NaN,NaN
1,Herr,Block,Philippe Camille Vincent,Architektur,2100,<p>Philippe Block is a structural engineer and...,http://block.arch.ethz.ch
2,Herr,Brandlhuber,Arno Hans,Architektur,2100,NaN,NaN
3,Herr,Caminada,Gion Antoni,Architektur,2100,NaN,NaN
4,Herr,Caruso,Adam,Architektur,2100,NaN,NaN


In [18]:
# producing single name column 
dep_df["Name"] = dep_df["FAMNAME"] + ", " + dep_df["VORNAME"]
# projecting into interesting columns 
dep_df = dep_df[["DEPARTEMENT_NAME","DEPARTEMENT_LEITZAHL","Name"]]
# renaming
dep_df.columns = ["department_name","department_code","name"]
dep_df.head(3)

,department_name,department_code,name
0,Architektur,2100,"Avermaete, Tom"
1,Architektur,2100,"Block, Philippe Camille Vincent"
2,Architektur,2100,"Brandlhuber, Arno Hans"


In [19]:
# merge as above
rc_m_3 = dep_df.merge(rc_m_2,how="outer",left_on="name",right_on="dc.contributor.author")
print(rc_m_3.shape)
rc_m_3.head(3)

(80597, 17)


,department_name,department_code,name,Professor,Org. unit code,Organisation,dc.contributor.author,dc.description.abstract,dc.title,id,dc.identifier.doi,dc.identifier.isbn,dc.identifier.issn,dc.identifier.olduri,dc.identifier.other,dc.identifier.uri,dc.identifier.wos
0,Architektur,2100.0,"Avermaete, Tom","Avermaete, Tom",9643.0,"Avermaete, Tom","Avermaete, Tom",NaN,The Travelling Type: How Buildings and Practic...,283758.0,NaN,978-1-4724-8504-5||978-1-315-58479-9,NaN,NaN,NaN,http://hdl.handle.net/20.500.11850/314867,NaN
1,Architektur,2100.0,"Avermaete, Tom","Avermaete, Tom",9643.0,"Avermaete, Tom","Avermaete, Tom",NaN,Beyond Innocence: The Norms and Forms of Colon...,284607.0,NaN,978-1-3156-1311-6||978-1-4724-8468-0||978-1-31...,NaN,NaN,10.4324/9781315613116,http://hdl.handle.net/20.500.11850/315490,NaN
2,Architektur,2100.0,"Avermaete, Tom","Avermaete, Tom",9643.0,"Avermaete, Tom","Avermaete, Tom",NaN,"Death of the Author, Center and Meta-Theory: E...",284605.0,NaN,978-1-1388-5698-1||978-1-3157-1899-6||978-1-31...,NaN,NaN,NaN,http://hdl.handle.net/20.500.11850/315497,NaN


In [20]:
# remove entries without a department
rc_m_4 = rc_m_3[rc_m_3['department_name'].notna()]
print(rc_m_4.shape)

(49197, 17)


### 2.3 Let's see what we get

In [21]:
# remove duplicates of documents (keep the first)
rc_m_5 = rc_m_4.drop_duplicates(subset = ['id'], keep = 'first')
print(rc_m_5.shape)

(41446, 17)


In [27]:
rc_m_5.to_csv("abstracts_research_incomplete.csv", index = False)

In [31]:
test = rc_m_5[['dc.identifier.doi','dc.identifier.isbn','dc.identifier.issn']]#,'dc.identifier.olduri',
               #'dc.identifier.other','dc.identifier.uri','dc.identifier.wos']]
test.head()
notna = test.dropna(thresh=1)
notna.shape

(33413, 3)

In [ ]:
rc_m_5[""]

In [22]:
# see how many of these have abstracts
rc_m_5[rc_m_5['dc.description.abstract'].notna()].shape[0]

6609

In [23]:
rc_m_abstract = rc_m_5[rc_m_5['dc.description.abstract'].notna()]
rc_m_abstract.head()

,department_name,department_code,name,Professor,Org. unit code,Organisation,dc.contributor.author,dc.description.abstract,dc.title,id,dc.identifier.doi,dc.identifier.isbn,dc.identifier.issn,dc.identifier.olduri,dc.identifier.other,dc.identifier.uri,dc.identifier.wos
10,Architektur,2100.0,"Avermaete, Tom","Avermaete, Tom",9643.0,"Avermaete, Tom","Avermaete, Tom",Objective: This article describes an approach ...,Evaluating the Psychosocial Impact of Indoor P...,303306.0,10.3929/ethz-b-000355341,NaN,NaN,NaN,10.1177/1937586718812439,http://hdl.handle.net/20.500.11850/355341,NaN
16,Architektur,2100.0,"Avermaete, Tom","Avermaete, Tom",9643.0,"Avermaete, Tom","Avermaete, Tom",In depictions of architects up to the 20th cen...,The Tools of the Architect: Towards a New Hist...,338321.0,10.3929/ethz-b-000392895,NaN,2050-5833,NaN,10.5334/ah.338,http://hdl.handle.net/20.500.11850/392895,NaN
98,Architektur,2100.0,"Dillenburger, Benjamin","Dillenburger, Benjamin",9566.0,"Dillenburger, Benjamin","Dillenburger, Benjamin","Although slabs are major concrete consumers, t...",Fast Complexity: Additive Manufacturing for Pr...,356306.0,NaN,978-3-030-49916-7||978-3-030-49915-0,2211-0844,NaN,10.1007/978-3-030-49916-7_102,http://hdl.handle.net/20.500.11850/426321,NaN
99,Architektur,2100.0,"Dillenburger, Benjamin","Dillenburger, Benjamin",9566.0,"Dillenburger, Benjamin","Dillenburger, Benjamin",The construction industry is a significant emi...,Geopolymer formulation for binder jet 3D printing,356309.0,NaN,978-3-030-49915-0||978-3-030-49916-7,2211-0844,NaN,10.1007/978-3-030-49916-7_16,http://hdl.handle.net/20.500.11850/426306,NaN
108,Architektur,2100.0,"Dillenburger, Benjamin","Dillenburger, Benjamin",9566.0,"Dillenburger, Benjamin","Dillenburger, Benjamin",Additive Manufacturing techniques enable an un...,Advances in Binder-Jet 3D Printing of Non-ceme...,356949.0,NaN,978-3-030-49915-0||978-3-030-49916-7,2211-0844,NaN,10.1007/978-3-030-49916-7_11,http://hdl.handle.net/20.500.11850/426777,NaN


In [26]:
rc_m_abstract[["department_name", "id"]].groupby("department_name").count()

,id
department_name,
Architektur,12
"Bau, Umwelt und Geomatik",461
Biologie,473
Biosysteme,296
Chemie und Angewandte Biowissenschaften,455
Erdwissenschaften,121
"Geistes-, Sozial- und Staatswissenschaften",186
Gesundheitswissenschaften und Technologie,428
Informatik,209
